In [3]:
import pandas as pd
import psycopg2

#establishing the connection
conn = psycopg2.connect(
    database='sqsight',
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

query = '''CREATE TABLE IF NOT EXISTS ingest (
           ingestfile VARCHAR(30) PRIMARY KEY, 
           istamp TIMESTAMP
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS issue (
            issueid CHAR(2) PRIMARY KEY,
            iname VARCHAR(50) NOT NULL
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS quarantine (
            quarid SERIAL PRIMARY KEY,
            ingestid VARCHAR(30),
            record TEXT NOT NULL
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS quarissue (
            qiid SERIAL PRIMARY KEY,
            quarid INT NOT NULL,
            issueid CHAR(2) NOT NULL
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS sighting (
            sightingid SERIAL PRIMARY KEY,
            squoutid INT NOT NULL,
            staproid CHAR(2) NOT NULL,
            quarid INT,
            ingestid VARCHAR(30),
            sightdate DATE,
            evidence VARCHAR(30),
            footsize SMALLINT
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS squout (
            squoutid SERIAL PRIMARY KEY,
            fname VARCHAR(50) NOT NULL,
            lname VARCHAR(50) NOT NULL
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS stapro (
            staproid CHAR(2) PRIMARY KEY,
            spname VARCHAR(50) NOT NULL,
            timezone VARCHAR(20)
        );'''

cursor.execute(query)
conn.commit()

In [5]:
import pandas as pd
df = pd.read_csv('ingest.csv')

for x in df.index:
    query = '''INSERT INTO ingest (ingestfile, istamp)
            VALUES ('%s', '%s')''' % (df['ingestfile'].loc[x], df['istamp'].loc[x])
    cursor.execute(query)
    conn.commit()


In [7]:
df = pd.read_csv('issue.csv')

for x in df.index:
    query = '''INSERT INTO issue (issueid, iname)
    VALUES ('%s', '%s')''' % (df['issueid'].loc[x], df['iname'].loc[x])
    cursor.execute(query)
    conn.commit()

In [9]:
df = pd.read_csv('sighting.csv')

for x in df.index:
    query = '''INSERT INTO sighting (squoutid, staproid, ingestid, sightdate, evidence, footsize)
    VALUES (%s, '%s', '%s', '%s', '%s', %s)''' % (df['squoutid'].loc[x], df['staproid'].loc[x],
                                                 df['ingestid'].loc[x], df['sightdate'].loc[x],
                                                 df['evidence'].loc[x], df['footsize'].loc[x])
    cursor.execute(query)
    conn.commit()

In [10]:
df = pd.read_csv('squout.csv')

for x in df.index:
    query = '''INSERT INTO squout (fname, lname)
    VALUES ('%s', '%s')''' % (df['fname'].loc[x], df['lname'].loc[x])
    
    cursor.execute(query)
    conn.commit()

In [13]:
df = pd.read_csv('stapro.csv')

for x in df.index:
    query = '''INSERT INTO stapro (staproid, spname, timezone)
    VALUES ('%s', '%s', '%s')''' % (df['staproid'].loc[x], df['name'].loc[x],
                                   df['timezone'].loc[x])
    
    cursor.execute(query)
    conn.commit()

In [15]:
query = '''SELECT *
        FROM sighting si, stapro sp, squout sq
        WHERE si.squoutid = sq.squoutid AND si.staproid = sp.staproid
        ORDER BY sightdate;'''

df = pd.read_sql(query, conn, index_col='sightingid')
print(df.head(10))

            squoutid staproid quarid       ingestid   sightdate evidence  \
sightingid                                                                 
1                 11       UT   None   someweek.csv  2023-06-10   Visual   
2                 19       WY   None   someweek.csv  2023-06-11   Visual   
3                  2       AB   None   someweek.csv  2023-06-11  Audible   
4                  2       OR   None   someweek.csv  2023-06-11    Photo   
5                 18       ID   None   someweek.csv  2023-06-12   Visual   
6                 24       AK   None   someweek.csv  2023-06-14   Visual   
7                 21       CA   None   someweek.csv  2023-06-14    Photo   
8                 11       AB   None   someweek.csv  2023-06-14  Audible   
9                 12       BC   None  otherweek.csv  2023-06-16    Video   
10                11       AB   None  otherweek.csv  2023-06-16  Audible   

            footsize staproid            spname  timezone  squoutid     fname  \
sighti

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_30307/957881540.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col='sightingid')
